In [1]:
import pandas as pd
df = pd.read_csv("Data/Hands-On-Gradient-Boosting-with-XGBoost-and-Scikit-learn/Chapter06/heart_disease.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [4]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [5]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [6]:
model = XGBClassifier(booster = "gbtree", objective = "binary:logistic",
                     random_state = 2)

In [7]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [8]:
scores = cross_val_score(model, X, y, cv = 5)

In [9]:
print("Accuracy : ", np.round(scores, 2))

Accuracy :  [0.84 0.85 0.82 0.8  0.77]


In [10]:
print("Mean Accuracy : {:.3f}".format(scores.mean()))

Mean Accuracy : 0.815


In [11]:
from sklearn.model_selection import StratifiedKFold

In [12]:
kfold = StratifiedKFold(n_splits= 5, shuffle=True, random_state=2)

In [13]:
scores = cross_val_score(model, X, y, cv= kfold)

In [14]:
print("Accuracy : ", np.round(scores, 2))

Accuracy :  [0.72 0.82 0.75 0.8  0.82]


In [15]:
print("Mean Accuracy : {:.3f}".format(scores.mean()))

Mean Accuracy : 0.782


In [16]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [18]:
def grid_search(params, random = False):
    xgb = XGBClassifier(booster = "gbtree", objective = "binary:logistic",
                     random_state = 2)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv= kfold, n_iter= 20, n_jobs= -1)
    else:
        grid = GridSearchCV(xgb, params, cv= kfold, n_jobs= - 1)
    grid.fit(X, y)
    best_params = grid.best_params_
    print("Best Params : ", best_params)
    best_score = grid.best_score_
    print("Training Score : {:.3f}".format(best_score))

In [20]:
grid_search(params= {"n_estimators" : [100, 200, 400, 800]})

Best Params :  {'n_estimators': 100}
Training Score : 0.782


In [21]:
grid_search(params= {'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

Best Params :  {'learning_rate': 0.05}
Training Score : 0.796


In [22]:
grid_search(params= {'max_depth' : [2,3,5,6,8]})

Best Params :  {'max_depth': 2}
Training Score : 0.799


In [23]:
grid_search(params= {'gamma' : [0, 0.1, 0.5, 1, 2, 5]})

Best Params :  {'gamma': 0.5}
Training Score : 0.796


In [24]:
grid_search(params= {'min_child_weight': [1, 2, 3, 4, 5]})

Best Params :  {'min_child_weight': 5}
Training Score : 0.812


In [25]:
grid_search(params= {'subsample': [0.5, 0.7, 0.8, 0.9, 1]})

Best Params :  {'subsample': 0.8}
Training Score : 0.796


In [26]:
grid_search(params= {'colsample_bytree': [0.5, 0.7, 0.8, 0.9, 1]})

Best Params :  {'colsample_bytree': 0.9}
Training Score : 0.799


## Early Stopping

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 2)

In [29]:
model = XGBClassifier(booster = "gbtree", objective = "binary:logistic", random_state = 2)

In [30]:
eval_set = [(X_test, y_test)]

In [31]:
eval_metric = 'error'

In [32]:
model.fit(X_train, y_train, eval_metric= eval_metric, eval_set= eval_set)

D:\xgb\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=2,
              reg_alpha=0, reg_lambda=1, ...)

In [33]:
y_pred = model.predict(X_test)

In [34]:
accuracy = accuracy_score(y_test, y_pred)

In [35]:
print("Accuracy : {:.3f}".format(accuracy))

Accuracy : 0.829


In [36]:
model = XGBClassifier(booster = "gbtree", objective = "binary:logistic", random_state = 2)
eval_set = [(X_test, y_test)]
eval_metric = 'error'
model.fit(X_train, y_train, eval_metric= eval_metric, eval_set= eval_set, early_stopping_rounds= 10, verbose= True)

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474


D:\xgb\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\xgb\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=2,
              reg_alpha=0, reg_lambda=1, ...)

In [37]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : {:.3f}".format(accuracy))

Accuracy : 0.895


In [38]:
model = XGBClassifier(booster = "gbtree", objective = "binary:logistic", random_state = 2, n_estimators = 5000)
eval_set = [(X_test, y_test)]
eval_metric = 'error'
model.fit(X_train, y_train, eval_metric= eval_metric, eval_set= eval_set, early_stopping_rounds= 100)

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

D:\xgb\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
D:\xgb\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[45]	validation_0-error:0.17105
[46]	validation_0-error:0.18421
[47]	validation_0-error:0.18421
[48]	validation_0-error:0.17105
[49]	validation_0-error:0.15789
[50]	validation_0-error:0.17105
[51]	validation_0-error:0.17105
[52]	validation_0-error:0.15789
[53]	validation_0-error:0.17105
[54]	validation_0-error:0.17105
[55]	validation_0-error:0.17105
[56]	validation_0-error:0.17105
[57]	validation_0-error:0.17105
[58]	validation_0-error:0.17105
[59]	validation_0-error:0.17105
[60]	validation_0-error:0.17105
[61]	validation_0-error:0.17105
[62]	validation_0-error:0.17105
[63]	validation_0-error:0.17105
[64]	validation_0-error:0.17105
[65]	validation_0-error:0.17105
[66]	validation_0-error:0.18421
[67]	validation_0-error:0.18421
[68]	validation_0-error:0.18421
[69]	validation_0-error:0.18421
[70]	validation_0-error:0.18421
[71]	validation_0-error:0.18421
[72]	validation_0-error:0.18421
[73]	validation_0-error:0.18421
[74]	validation_0-error:0.17105
[75]	validation_0-error:0.18421
[76]	val

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=5000,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=2,
              reg_alpha=0, reg_lambda=1, ...)

In [39]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy : {:.3f}".format(accuracy))

Accuracy : 0.895


In [41]:
grid_search(params= {'n_estimators' : [2, 25, 50, 75, 100]})

Best Params :  {'n_estimators': 50}
Training Score : 0.789


In [43]:
grid_search(params = {'max_depth' : [1, 2, 3, 4, 5, 6, 7, 8], 'n_estimators': [50]})

Best Params :  {'max_depth': 1, 'n_estimators': 50}
Training Score : 0.839


In [44]:
grid_search(params = {'max_depth' : [1, 2, 3, 4, 5, 6, 7, 8], 
                      'n_estimators': [2, 50, 100]})

Best Params :  {'max_depth': 1, 'n_estimators': 50}
Training Score : 0.839


In [45]:
grid_search(params = {'learning_rate' : [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
                     'max_depth' : [1],
                     'n_estimators' : [50]})

Best Params :  {'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50}
Training Score : 0.839


In [46]:
grid_search(params = {'min_child_weight' : [1,2,3,4,5],
                     'max_depth' : [1],
                     'n_estimators' : [50]})

Best Params :  {'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 50}
Training Score : 0.839


In [47]:
grid_search(params = {'subsample' : [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                     'max_depth' : [1],
                     'n_estimators' : [50]})

Best Params :  {'max_depth': 1, 'n_estimators': 50, 'subsample': 1.0}
Training Score : 0.839


In [48]:
grid_search(params = {'learning_rate' : [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
                      'subsample' : [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                      'min_child_weight' : [1, 2, 3, 4, 5],
                     'max_depth' : [1, 2, 3, 4, 5],
                     'n_estimators' : [2]})

Best Params :  {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 2, 'subsample': 0.9}
Training Score : 0.812


In [49]:
grid_search(params = {'learning_rate' : [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
                      'subsample' : [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                      'min_child_weight' : [1, 2, 3, 4, 5],
                     'max_depth' : [1, 2, 3, 4, 5, None],
                     'n_estimators' : [2, 25, 50, 75, 100]},
           random= True)

Best Params :  {'subsample': 1.0, 'n_estimators': 100, 'min_child_weight': 4, 'max_depth': 1, 'learning_rate': 0.3}
Training Score : 0.816


In [50]:
grid_search(params= {'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                    'max_depth' : [1],
                    'n_estimators': [50]})

Best Params :  {'colsample_bytree': 0.8, 'max_depth': 1, 'n_estimators': 50}
Training Score : 0.842


In [51]:
grid_search(params= {'colsample_bylevel': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                    'max_depth' : [1],
                    'n_estimators': [50]})

Best Params :  {'colsample_bylevel': 0.8, 'max_depth': 1, 'n_estimators': 50}
Training Score : 0.842


In [52]:
grid_search(params= {'colsample_bylevel': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                     'colsample_bynode': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                    'max_depth' : [1],
                    'n_estimators': [50]})

Best Params :  {'colsample_bylevel': 0.6, 'colsample_bynode': 0.8, 'colsample_bytree': 0.7, 'max_depth': 1, 'n_estimators': 50}
Training Score : 0.852


In [53]:
grid_search(params= {'gamma' : [0, 0.1, 0.05, 0.1, 0.5, 1, 2, 3],
                    'colsample_bynode':[0.8],
                    'colsample_bylevel':[0.6],
                    'colsample_bytree':[0.7],
                    'max_depth':[1],
                    'n_estimators':[50]})

Best Params :  {'colsample_bylevel': 0.6, 'colsample_bynode': 0.8, 'colsample_bytree': 0.7, 'gamma': 0, 'max_depth': 1, 'n_estimators': 50}
Training Score : 0.852
